In [1]:
# Importing Required modules
import pandas as pd
import mysql.connector as msql
import math
import os
import datetime
import re
import requests

In [2]:
# Displaying all columns and rows
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [3]:
#Reading She for STEM Incubator file present on source files
directory_path =(r"C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\04 Incubator - Curriculum Cohort\Incubator 6.0 UK\data_files\02_Source_Kalpana Incubator")
csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

for file in csv_files:
    file_path = os.path.join(directory_path, file)
    data = pd.read_csv(file_path)
    print(f"Data from {file}:")

Data from She for STEM - Uttarakhand.csv:


In [5]:
# Define a list of column names to extract for putting into database 
columns_to_extract = ['Name', 'Email', 'Segment', 'Mobile', 'Enroll Date', 'Assigned Through', 'Course Name','Start Date','Last Login']

# Use the loc method to extract the specified columns
df = data.loc[:, columns_to_extract]


In [7]:
# Create a new dataframe with 'email' column and columns that start with 'Week' 'Video', 'Recording', and 'Master class'
data = data[['Email'] + [col for col in data.columns if col.startswith(('Week','Video', 'SUK', 'Workshop', 'Master Class'))]]


In [8]:
# Copy the columns of data dataframe in oldcol
oldcol = list(data.columns)


In [9]:
#oldcol =data.columns

In [10]:
oldcol

['Email',
 'Week 0 : Overview - Program Kalpana - 6pgx1c40pq',
 'Video ! : Why women in STEM. What has Kalpana program achieved until now? - 62c7d78d0cf273488220b8d4',
 'Video @ : What do we expect from our Kalpana fellows? - 62c7d10e0cf273488220b5bf',
 'SUK + : Program Inauguration & Icebreaker 7 Sept - 66dc3e605ae308684ff5e726',
 'Week 1 : Setting Aspirational Goals - ng1lgwrxx4',
 'Video # : Goal setting - Part 1 - 6529ffc5e4b0a564951cb119',
 'Video $ : Goal setting - Part 2 - 6529ffd4e4b08a0cfb8d0628',
 'Workshop @ : Setting Aspirational Goals, Thursday, Sept 12, 3-4 PM - 66de9bf79027bf0127e793ec',
 'Master Class # : Searching and Securing Internships - Friday, Sept 13, 3-4 PM - 66e125d35e916d19ae3df9b1',
 'Week 2 : Reflect on Setting Goals - b1jsvbvfh1',
 'SUK ! : Speak Up Kalpana session, Friday, Sept 20, 3-4 PM - 66e13e6e91e40e365df6fdd4',
 'Week 3 : Setting SMART goals - 19n1ky6p5n',
 'Video % : What is SMART goal ? - 62b3fded0cf2f5120a60c932',
 'Video ^ : What is the differenc

In [11]:
# Cheaking shape of our dataset
data.shape

(503, 48)

In [12]:
# Initialize variables to keep track of the current week, video count, recording count, and master class count
week_col = None
video_count = 0
recording_count = 0
master_count = 0
workshop_count = 0

# Create an empty list to store the new column names
new_cols = []

# Iterate over each column in the data
for col in data.columns:
    # If the column starts with 'Week'
    if col.startswith('Week'):
        # Split the column name by space and get the second element (the week number)
        week_col = col.split()[1]
        # Reset the video, recording, and master class counts for the new week
        video_count = 0
        recording_count = 0
    # If the column starts with 'Video'
    elif col.startswith('Video'):
        # Increment the video count for the current week
        video_count += 1
        # Append a new column name to the list using f-string formatting
        new_cols.append(f'WK{week_col}_V{video_count}')
    # If the column starts with 'Workshop'
    elif col.startswith('Workshop'):
        # Increment the master class count for the current week
        workshop_count += 1
        # Append a new column name to the list using f-string formatting
        new_cols.append(f'WK{week_col}_WS{workshop_count}')
    # If the column doesn't start with any of the above prefixes
    # If the column starts with 'Recording'
    elif col.startswith('SUK'):
        # Increment the recording count for the current week
        recording_count += 1
        # Append a new column name to the list using f-string formatting
        new_cols.append(f'WK{week_col}_SUK_V')
    # If the column starts with 'Master Class'
    elif col.startswith('Master Class'):
        # Increment the master class count for the current week
        master_count += 1
        # Append a new column name to the list using f-string formatting
        new_cols.append(f'WK{week_col}_Master{master_count}')
    # If the column doesn't start with any of the above prefixes
    else:
        # Append the original column name to the list
        new_cols.append(col)

# Remove all columns that start with 'Week' from the data
data = data.loc[:, ~data.columns.str.startswith('Week')]
# Assign the new column names to the data
data.columns = new_cols


In [13]:
# Rechecking the shape of dataframe
data.shape

(503, 37)

In [14]:
# Cheaking columns name
data.columns

Index(['Email', 'WK0_V1', 'WK0_V2', 'WK0_SUK_V', 'WK1_V1', 'WK1_V2', 'WK1_WS1',
       'WK1_Master1', 'WK2_SUK_V', 'WK3_V1', 'WK3_V2', 'WK3_WS2',
       'WK3_Master2', 'WK4_SUK_V', 'WK4_WS3', 'WK5:_V1', 'WK5:_V2', 'WK5:_V3',
       'WK5:_WS4', 'WK5:_WS5', 'WK5:_Master3', 'WK6_WS6', 'WK6_SUK_V',
       'WK7_V1', 'WK7_WS7', 'WK9_V1', 'WK9_V2', 'WK9_WS8', 'WK9_Master4',
       'WK10:_V1', 'WK10:_V2', 'WK10:_V3', 'WK10:_V4', 'WK10:_WS9',
       'WK10:_Master5', 'WK10:_WS10', 'WK10:_SUK_V'],
      dtype='object')

In [15]:
# To rename duplicate column names correctly
def rename_duplicate_columns(data):
    seen = {}
    new_columns = []
    
    for col in data.columns:
        if col not in seen:
            seen[col] = 1
            new_columns.append(col)
        else:
            seen[col] += 1
            new_columns.append(f"{col}{seen[col]}")
    
    data.columns = new_columns
    return data

# Applying the function
data = rename_duplicate_columns(data)

In [16]:
# Rename columns starting with 'WKNone' to 'WK0' and removing : if present
data.columns = [
    col.replace('WKNone', 'WK0').replace(':', '') if col.startswith('WKNone') or ':' in col else col
    for col in data.columns]

In [17]:
#data.rename(columns = {'WK10_SUK_V': 'WK10_SUK_V1'}, inplace = True)

In [18]:
# Cheaking for duplicate data
data.duplicated().sum()

0

In [19]:
# Locating columns for extracting only numbers
col1 = data.iloc[:,1:]

In [20]:
col1.dtypes


WK0_V1           object
WK0_V2           object
WK0_SUK_V        object
WK1_V1           object
WK1_V2           object
WK1_WS1          object
WK1_Master1      object
WK2_SUK_V        object
WK3_V1           object
WK3_V2           object
WK3_WS2          object
WK3_Master2      object
WK4_SUK_V        object
WK4_WS3          object
WK5_V1           object
WK5_V2           object
WK5_V3           object
WK5_WS4          object
WK5_WS5          object
WK5_Master3      object
WK6_WS6          object
WK6_SUK_V        object
WK7_V1           object
WK7_WS7          object
WK9_V1           object
WK9_V2           object
WK9_WS8          object
WK9_Master4      object
WK10_V1          object
WK10_V2          object
WK10_V3          object
WK10_V4          object
WK10_WS9         object
WK10_Master5     object
WK10_WS10       float64
WK10_SUK_V      float64
dtype: object

In [21]:
# Code for extracting only numbers from dataset
for column in [i for i in col1.columns if col1[i].dtype == 'object']:
    data[column] = data[column].astype(str).str.extract('(\d+)').astype(float)

In [22]:
# Filling Null values with zero
fillna = data.iloc[:,1:]
fillnacol=fillna.columns
data[fillnacol]=data[fillnacol].fillna(0)

In [24]:
data.columns

Index(['Email', 'WK0_V1', 'WK0_V2', 'WK0_SUK_V', 'WK1_V1', 'WK1_V2', 'WK1_WS1',
       'WK1_Master1', 'WK2_SUK_V', 'WK3_V1', 'WK3_V2', 'WK3_WS2',
       'WK3_Master2', 'WK4_SUK_V', 'WK4_WS3', 'WK5_V1', 'WK5_V2', 'WK5_V3',
       'WK5_WS4', 'WK5_WS5', 'WK5_Master3', 'WK6_WS6', 'WK6_SUK_V', 'WK7_V1',
       'WK7_WS7', 'WK9_V1', 'WK9_V2', 'WK9_WS8', 'WK9_Master4', 'WK10_V1',
       'WK10_V2', 'WK10_V3', 'WK10_V4', 'WK10_WS9', 'WK10_Master5',
       'WK10_WS10', 'WK10_SUK_V'],
      dtype='object')

## Merging data from Live Session Attendance Sheet

In [ ]:
# URL of the Google Sheets document

url = "" # Enter Google Sheet Link here

# The URL needs to be modified to point to the export format

url = url.replace('/edit?usp=sharing', '/export?format=xlsx')

# Read the data from the URL into a pandas DataFrame

la = pd.read_excel(url)


In [27]:
# Get the columns to keep
columns_to_keep = [col for col in la.columns if col.startswith("Timestamp") or col.startswith("Enter the email ID that you use to login to the VigyanShaala app/platform") ]

# Drop the columns not in columns_to_keep
la.drop(columns=[col for col in la.columns if col not in columns_to_keep], inplace=True)


In [28]:
if 'Enter the email ID that you use to login to the VigyanShaala app/platform' in la.columns: la.rename(columns={'Enter the email ID that you use to login to the VigyanShaala app/platform': 'Email'}, inplace=True)

In [29]:
# Convert Timestamp column to datetime
la['Timestamp'] = pd.to_datetime(la['Timestamp'])

# Create a new column 'dates' with the desired format
la['Date'] = la['Timestamp'].dt.strftime('%Y-%m-%d')

In [30]:
# Drop the Timestamp column
la = la.drop(columns=['Timestamp'])

###  👇 The date and the Name of session should be same so it will get merge 👇

In [31]:
# Reading Excel file form our source
ld  = pd.read_excel(r"C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\04 Incubator - Curriculum Cohort\Incubator 6.0 UK\data_files\00_Source_Kalpana Live Session Info\Live Session Dates.xlsx")

In [32]:
ld.head()

,Date,Sessions
0,2024-09-07,WK0_SUK_V
1,2024-09-12,WK1_WS1
2,2024-09-13,WK1_Master1
3,2024-09-20,WK2_SUK_V
4,2024-09-26,WK3_WS2


In [33]:
# Convert 'Date' column to datetime format
la['Date'] = pd.to_datetime(la['Date'])
ld['Date'] = pd.to_datetime(ld['Date'])

# Merge dataframes on 'Date'
md = pd.merge(la, ld, on='Date', how='outer')


In [34]:
# Drop rows where 'Email' column has null values
md = md.dropna(subset=['Email'])

In [35]:
md.columns

Index(['Email', 'Date', 'Sessions'], dtype='object')

In [36]:
# Function to clean email addresses
def clean_email(email):

    # Convert to lowercase and remove extra spaces
    cleaned_email = email.lower().strip()
    # Remove patterns like ".com.1"
    cleaned_email = re.sub(r'\.com\.\d+', '.com', cleaned_email)
    return cleaned_email

# Apply the function to the 'email' column
md['Email'] = md['Email'].apply(clean_email)

In [37]:
# Remove the 'name' column
md = md.drop(columns=['Date'])

# Add a new column 'duration' with all values set to 3600
md['Duration'] = 3600

In [38]:
md_pivot = md.pivot_table(index='Email', columns='Sessions')

In [39]:
md_pivot.columns = [f'{col[0]} - {col[1]}' for col in md_pivot.columns]


In [40]:
md_pivot.reset_index(inplace=True)

In [41]:
# Renaming columns of dataframe
md_pivot.columns = md_pivot.columns.str.replace(r'Duration - ', '')


In [42]:
# Merge dataframes on 'Email' column
merged_df = pd.merge(data, md_pivot, on='Email', how='outer', suffixes=('_df', '_da'))

# List of columns to sum
columns_to_sum = [col for col in md_pivot.columns if col != 'Email']

# Sum the matching columns
for col in columns_to_sum:
    merged_df[col] = merged_df[f'{col}_df'].fillna(0) + merged_df[f'{col}_da'].fillna(0)

# Drop the intermediate columns
merged_df.drop(columns=[f'{col}_df' for col in columns_to_sum] + [f'{col}_da' for col in columns_to_sum], inplace=True)



In [45]:
# Ensure that merged_df contains all columns from gd
missing_cols = [col for col in data.columns if col not in merged_df.columns]
for col in missing_cols:
    merged_df[col] = pd.NA

# Reorder the columns of merged_df to match the order of gd.columns
merged_df = merged_df[data.columns]

# Enroll Date MySQL Table

In [46]:
# Creating new table which for Enroll_Dates which is taken from incubator graphy sheet
Enroll=pd.DataFrame(df[["Email", 'Enroll Date']])

In [47]:
# Extracting only Enroll_Date
Enroll[['Enroll Date','Time']]=Enroll['Enroll Date'].str.split(' ',expand=True)
Enroll=Enroll.drop(["Time"],axis=1)

In [ ]:
# Connecting to MySQL Database
conn= msql.connect(host='',user='',password="@123",database="",auth_plugin='')
cursor =conn.cursor()

In [49]:
# Inserting data to 03_enroll_date table
for i,row in Enroll.iterrows():
    cursor.execute("insert IGNORE into 03_enroll_date (Email,Incubator) values(%s,%s)",tuple(row))

In [50]:
# Replacing data  with new data to 03_enroll_date table
for i,row in Enroll.iterrows():
    cursor.execute("REPLACE into 03_enroll_date (Email,incubator) values(%s,%s)",tuple(row))

In [51]:
conn.commit()

# Payment details MySQL table

In [52]:
# Creating new table which for Payment which is taken from incubator graphy sheet
Payment=pd.DataFrame(df[["Email","Assigned Through"]])

# Extracting only Payment removing Order Id
Payment[["Assigned Through","Order_ID"]]=Payment["Assigned Through"].str.split("-",expand=True)
Payment.drop(["Order_ID"],axis=1,inplace=True)

In [53]:
# Assigning fee cost to different cathegory of enrollment
Payment["Assigned Through"]=Payment["Assigned Through"].replace(['Admin'],'0')
Payment["Assigned Through"]=Payment["Assigned Through"].replace(['Excel Upload'],'0')
Payment["Assigned Through"]=Payment["Assigned Through"].replace(['Paid Transaction '],'1899')

In [ ]:
# Connecting to MySQL Database
conn= msql.connect(host='',user='',password="",database="",auth_plugin='')
cursor =conn.cursor()

In [55]:
# Inserting data into 02_payment_details 
for i,row in Payment.iterrows():
    cursor.execute("insert Ignore into 02_payment_details (Email,Incubator_Fee) values(%s,%s)",tuple(row))

In [56]:
# Replacing data with new data into 02_payment_details 
for i,row in Payment.iterrows():
    cursor.execute("REPLACE into 02_payment_details (Email,Incubator_Fee) values(%s,%s)",tuple(row))

In [57]:
conn.commit()

# Batch Lastlogin Startdate MySQL Table

In [58]:
# Creating new table which for Enroll_Dates which is taken from incubator graphy sheet
Batch=pd.DataFrame(df[["Email", 'Last Login','Start Date']])

In [59]:
# Extracting only Enroll_Date
Batch[['Last Login','Time']]=Batch['Last Login'].str.split(' ',expand=True)
Batch=Batch.drop(["Time"],axis=1)

In [60]:
# Function to convert 'Start Date' format
def convert_start_date(date_str):
    if re.match(r'\d{4}/\d{2}/\d{2}', date_str):
        return pd.to_datetime(date_str, format='%Y/%m/%d').strftime('%d-%m-%Y')
    return date_str

# Function to convert 'Last Login' format
def convert_last_login(date_str):
    if re.match(r'\d{2}/\d{2}/\d{2}', date_str):
        return pd.to_datetime(date_str, format='%d/%m/%y').strftime('%d-%m-%Y')
    return date_str


# Apply the conversion functions to the respective columns
Batch['Start Date'] = Batch['Start Date'].apply(convert_start_date)
Batch['Last Login'] = Batch['Last Login'].apply(convert_last_login)

In [ ]:
# Connecting to MySQL Database
conn= msql.connect(host='',user='',password="",database="",auth_plugin='')
cursor =conn.cursor()

In [62]:
# Inserting data to 15_batch_lastlogin_startdate table
for i,row in Batch.iterrows():
    cursor.execute("insert IGNORE into 15_batch_lastlogin_startdate (Email,Incubator_Last_Login,Incubator_Start_Date) values(%s,%s,%s)",tuple(row))

In [63]:
# Replacing data  with new data to 03_enroll_date table
for i,row in Batch.iterrows():
    cursor.execute("REPLACE into 15_batch_lastlogin_startdate (Email,Incubator_Last_Login,Incubator_Start_Date) values(%s,%s,%s)",tuple(row))

In [64]:
conn.commit()

# Changing seconds to Percentage

In [67]:
# Saving previous column name 
newcol = list(merged_df.columns)


In [68]:
newcol

['Email',
 'WK0_V1',
 'WK0_V2',
 'WK0_SUK_V',
 'WK1_V1',
 'WK1_V2',
 'WK1_WS1',
 'WK1_Master1',
 'WK2_SUK_V',
 'WK3_V1',
 'WK3_V2',
 'WK3_WS2',
 'WK3_Master2',
 'WK4_SUK_V',
 'WK4_WS3',
 'WK5_V1',
 'WK5_V2',
 'WK5_V3',
 'WK5_WS4',
 'WK5_WS5',
 'WK5_Master3',
 'WK6_WS6',
 'WK6_SUK_V',
 'WK7_V1',
 'WK7_WS7',
 'WK9_V1',
 'WK9_V2',
 'WK9_WS8',
 'WK9_Master4',
 'WK10_V1',
 'WK10_V2',
 'WK10_V3',
 'WK10_V4',
 'WK10_WS9',
 'WK10_Master5',
 'WK10_WS10',
 'WK10_SUK_V']

In [69]:
# Saving previous column name for data1 by removing "Email column"
merged_df_columns = list(merged_df.columns)

# Exclude the 'email' column if it exists
if 'Email' in merged_df_columns:
    merged_df_columns.remove('Email')

merged_df1col = merged_df_columns




In [70]:
# Remove columns starting with "Week" from the list of old columns
oldcol = [col for col in oldcol if not col.startswith("Week")]

# Determine the number of columns to rename (minimum of available columns)
num_columns_to_rename = min(len(merged_df.columns), len(oldcol))

# Rename columns using a dictionary mapping
merged_df.rename(columns=dict(zip(merged_df.columns[:num_columns_to_rename], oldcol[:num_columns_to_rename])), inplace=True)

# Remove any additional information after the colon in column names
merged_df.columns = merged_df.columns.str.split(':').str[0]

# Remove leading and trailing spaces from column names
merged_df.columns = merged_df.columns.str.strip()


In [71]:
merged_df.columns

Index(['Email', 'Video !', 'Video @', 'SUK +', 'Video #', 'Video $',
       'Workshop @', 'Master Class #', 'SUK !', 'Video %', 'Video ^',
       'Workshop $', 'Master Class $', 'SUK @', 'Workshop !!', 'Video $?',
       'Video %+', 'Video %!', 'Workshop !#', 'Workshop %', 'Master Class !',
       'Workshop ^', 'SUK #', 'Video %@', 'Workshop &', 'Video %#', 'Video %$',
       'Workshop *', 'Master Class %', 'Video &', 'Video *', 'Video ?',
       'Video !+', 'Workshop ?', 'Master Class !^', 'Workshop !@', 'SUK $'],
      dtype='object')

In [72]:
def rename_column(col):
    # Split the column name by space
    parts = col.split()
    special_chars = {'!': '1', '@': '2', '#': '3', '$': '4', '%': '5', '^': '6', '&': '7', '*': '8', '?': '9', '+': '0'}
    
    # If the first part is 'SUK', add the number corresponding to the last character
    if parts[0] == 'SUK':
        return parts[0] + special_chars.get(parts[1][-1], '0')
    
    # If the first part is 'Video', use the prefix 'VID' and the numbers corresponding to the last two characters
    elif parts[0] == 'Video':
        last_char = parts[1][-1]
        second_last_char = parts[1][-2] if len(parts[1]) > 1 else '+'
        return 'VID' + special_chars.get(second_last_char, '0') + special_chars.get(last_char, '0')

    # If the first part is 'Workshop', use the prefix 'VID' and the numbers corresponding to the last two characters
    elif parts[0] == 'Workshop':
        last_char = parts[1][-1]
        second_last_char = parts[1][-2] if len(parts[1]) > 1 else '+'
        return 'WS' + special_chars.get(second_last_char, '0') + special_chars.get(last_char, '0')
    
    # If the first part contains 'Master' and 'Class' is present, use the prefix 'MC' and the numbers corresponding to the last two characters
    elif 'Master' in parts and 'Class' in parts:
        index = parts.index('Master')
        last_char = parts[index + 2][-1]
        second_last_char = parts[index + 2][-2] if len(parts[index + 2]) > 1 else '+'
        return 'MC' + special_chars.get(second_last_char, '0') + special_chars.get(last_char, '0')
    
    # Otherwise, return the column name as it is
    else:
        return col


In [73]:
# Rename the column names using the rename_column function
merged_df.rename(columns=rename_column, inplace=True)

In [74]:
merged_df.columns

Index(['Email', 'VID01', 'VID02', 'SUK0', 'VID03', 'VID04', 'WS02', 'MC03',
       'SUK1', 'VID05', 'VID06', 'WS04', 'MC04', 'SUK2', 'WS11', 'VID49',
       'VID50', 'VID51', 'WS13', 'WS05', 'MC01', 'WS06', 'SUK3', 'VID52',
       'WS07', 'VID53', 'VID54', 'WS08', 'MC05', 'VID07', 'VID08', 'VID09',
       'VID10', 'WS09', 'MC16', 'WS12', 'SUK4'],
      dtype='object')

In [ ]:
# Reading Excel file form our source
excel_file  = pd.read_excel(r"")

In [76]:
merged_df1=pd.DataFrame()

In [77]:
# Iterate over each column in the 'data' DataFrame
for column_name in merged_df.columns:
    # Check if the column exists in the Excel file
    if column_name in excel_file['Code'].values:
        # Get the corresponding value in the "Time" column from the Excel file
        value = excel_file.loc[excel_file['Code'] == column_name, 'Time'].values[0]
        
        # Calculate the percentage value
        percentage_value = (merged_df[column_name] * 100) / value
        
        # Assign the calculated percentage value to a new column in the "merged_df1" DataFrame with the same name as the "Code" column
        merged_df1[column_name] = percentage_value
    else:
        # Handle the case when the column is present in 'merged_df' but not in the Excel file
        print(f"Column '{column_name}' present in 'merged_df' but not in the Excel file. Skipping.")


Column 'Email' present in 'merged_df' but not in the Excel file. Skipping.


In [78]:
# Iterate over each row in the Excel file
for index, row in excel_file.iterrows():
    # Get the value in the "Code" column
    column_name = row['Code']
    
    # Get the value in the "Time" column
    value = row['Time']
    
    # Check if the 'Code' column exists in the 'merged_df' DataFrame
    if column_name in merged_df.columns:
        # Calculate the percentage value
        percentage_value = (merged_df[column_name] * 100) / value
        
        # Assign the calculated percentage value to a new column in the "merged_df1" DataFrame with the same name as the "Code" column
        merged_df1[column_name] = percentage_value

In [80]:
# Changing column name back to new column
merged_df.columns = newcol 
merged_df1.columns = merged_df1col

In [81]:
merged_df.columns

Index(['Email', 'WK0_V1', 'WK0_V2', 'WK0_SUK_V', 'WK1_V1', 'WK1_V2', 'WK1_WS1',
       'WK1_Master1', 'WK2_SUK_V', 'WK3_V1', 'WK3_V2', 'WK3_WS2',
       'WK3_Master2', 'WK4_SUK_V', 'WK4_WS3', 'WK5_V1', 'WK5_V2', 'WK5_V3',
       'WK5_WS4', 'WK5_WS5', 'WK5_Master3', 'WK6_WS6', 'WK6_SUK_V', 'WK7_V1',
       'WK7_WS7', 'WK9_V1', 'WK9_V2', 'WK9_WS8', 'WK9_Master4', 'WK10_V1',
       'WK10_V2', 'WK10_V3', 'WK10_V4', 'WK10_WS9', 'WK10_Master5',
       'WK10_WS10', 'WK10_SUK_V'],
      dtype='object')

In [82]:
merged_df1.columns

Index(['WK0_V1', 'WK0_V2', 'WK0_SUK_V', 'WK1_V1', 'WK1_V2', 'WK1_WS1',
       'WK1_Master1', 'WK2_SUK_V', 'WK3_V1', 'WK3_V2', 'WK3_WS2',
       'WK3_Master2', 'WK4_SUK_V', 'WK4_WS3', 'WK5_V1', 'WK5_V2', 'WK5_V3',
       'WK5_WS4', 'WK5_WS5', 'WK5_Master3', 'WK6_WS6', 'WK6_SUK_V', 'WK7_V1',
       'WK7_WS7', 'WK9_V1', 'WK9_V2', 'WK9_WS8', 'WK9_Master4', 'WK10_V1',
       'WK10_V2', 'WK10_V3', 'WK10_V4', 'WK10_WS9', 'WK10_Master5',
       'WK10_WS10', 'WK10_SUK_V'],
      dtype='object')

In [83]:
# Creating a function "Max_Value" for converting % max 100 if values are above 100
def Max_Value(value):
    if value >=100:
        return 100
    else:
        return value

In [84]:
# Applying Max_Value function
per1=merged_df1.iloc[:,0:]
per1c=per1.columns
merged_df1[per1c]=merged_df1[per1c].applymap(Max_Value)

C:\Users\spjay\AppData\Local\Temp\ipykernel_44632\3614295319.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  merged_df1[per1c]=merged_df1[per1c].applymap(Max_Value)


# Code for converting seconds to hours

In [85]:
# Creating a function for converting seconds to hours
def Convert_Hours(seconds):
    hours = seconds / (3600)
    return hours
n = 5400
print(Convert_Hours(n))


1.5


In [86]:
# Applying Convert_Hours function
col1 = merged_df.iloc[:,1:]
col=col1.columns
merged_df[col]=merged_df[col].apply(Convert_Hours)

# Code for Pre Recorded Videos: Total hours, Percentage

# 👇 Input Weeks Here 👇

In [87]:
# Define the week number up to which you want to include columns
end_week = 10

In [88]:
# Create a list of all column names up to the end week
cols_to_select = []
for week in range(end_week + 1):
    for video in range(1, 10):
        col_name = f'WK{week}_V{video}'
        if col_name in merged_df.columns:
            cols_to_select.append(col_name)

# Select the desired columns and create a new DataFrame
Recorded_Total = merged_df[cols_to_select]

In [90]:
merged_df["Pre_Recorded_Total_Hours"]=Recorded_Total.sum(axis=1)

In [91]:
# Create a list of all column names up to the end week
cols_to_select_per = []
for week in range(end_week + 1):
    for video in range(1, 10):
        col_name = f'WK{week}_V{video}'
        if col_name in merged_df1.columns:
            cols_to_select_per.append(col_name)

# Select the desired columns and create a new DataFrame
Recorded_Percent = merged_df1[cols_to_select_per]

In [93]:
merged_df["Pre_Recorded_Percentage"]=Recorded_Percent.mean(axis=1)

# Code for SUK Videos: Total hours & Percentage

In [94]:
# Create a list of all column names up to the end week
cols_to_select = []
for week in range(end_week + 1):
    col_name = f'WK{week}_SUK_V'
    if col_name in merged_df.columns:
        cols_to_select.append(col_name)

# Select the desired columns and create a new DataFrame
SUK_Recorded_Total = merged_df[cols_to_select]

In [96]:
merged_df["SUK_Total_Hours"]=SUK_Recorded_Total.sum(axis=1)

In [97]:
# Create a list of all column names up to the end week
cols_to_select = []
for week in range(end_week + 1):
    col_name = f'WK{week}_SUK_V'
    if col_name in merged_df1.columns:
        cols_to_select.append(col_name)

# Select the desired columns and create a new DataFrame
SUK_Recorded_Percent = merged_df1[cols_to_select]

In [98]:
SUK_Recorded_Percent.fillna(0, inplace=True)
# Remove columns with all zero values
SUK_Recorded_Percent = SUK_Recorded_Percent.loc[:, (SUK_Recorded_Percent != 0).any(axis=0)]

C:\Users\spjay\AppData\Local\Temp\ipykernel_44632\4141890018.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SUK_Recorded_Percent.fillna(0, inplace=True)


In [99]:
# Compute the mean, excluding columns that contain all zeros
merged_df["SUK_Percentage"]=SUK_Recorded_Percent.mean(axis=1)

# Code for Master Class: Total hours & Percentage

In [100]:
# Create a list of all column names up to the end week
cols_to_select = []
for week in range(end_week + 2):
    for masterclass in range(1, 50):
        col_name = f'WK{week}_Master{masterclass}'
        if col_name in merged_df.columns:
            cols_to_select.append(col_name)

# Select the desired columns and create a new DataFrame
Masterclass_Total = merged_df[cols_to_select]


In [101]:
merged_df["Masterclass_Total_Hours"]=Masterclass_Total.sum(axis=1)

In [102]:
# Create a list of all column names up to the end week
cols_to_select = []
for week in range(end_week + 1):
    for masterclass in range(1, 10):
        col_name = f'WK{week}_Master{masterclass}'
        if col_name in merged_df1.columns:
            cols_to_select.append(col_name)

# Select the desired columns and create a new DataFrame
Masterclass_Percent = merged_df1[cols_to_select]


In [104]:
Masterclass_Percent.fillna(0, inplace=True)
# Remove columns with all zero values
Masterclass_Percent = Masterclass_Percent.loc[:, (Masterclass_Percent != 0).any(axis=0)]

C:\Users\spjay\AppData\Local\Temp\ipykernel_44632\1041955373.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Masterclass_Percent.fillna(0, inplace=True)


In [105]:
# Calculate the mean and assign it to the new column
merged_df["Masterclass_Percentage"] = Masterclass_Percent.mean(axis=1)


# Code for Workshop Class: Total hours & Percentage

In [106]:
# Create a list of all column names up to the end week
cols_to_select = []
for week in range(end_week + 1):
    for video in range(1, 10):
        col_name = f'WK{week}_WS{video}'
        if col_name in merged_df.columns:
            cols_to_select.append(col_name)

# Select the desired columns and create a new DataFrame
WS_Total = merged_df[cols_to_select]

In [107]:
merged_df["Workshop_Total_Hours"]=WS_Total.sum(axis=1)

In [108]:
# Create a list of all column names up to the end week
cols_to_select_per = []
for week in range(end_week + 1):
    for video in range(1, 10):
        col_name = f'WK{week}_WS{video}'
        if col_name in merged_df1.columns:
            cols_to_select_per.append(col_name)

# Select the desired columns and create a new DataFrame
WS_Percent = merged_df1[cols_to_select_per]

In [109]:
WS_Percent.fillna(0, inplace=True)
# Remove columns with all zero values
WS_Percent = WS_Percent.loc[:, (WS_Percent != 0).any(axis=0)]

C:\Users\spjay\AppData\Local\Temp\ipykernel_44632\2660094332.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WS_Percent.fillna(0, inplace=True)


In [110]:
# Compute the mean, excluding columns that contain all zeros
merged_df["Workshop_Percentage"]=WS_Percent.mean(axis=1)

# Code for Program: Total hours & Percentage

In [111]:
# select columns from data where the column names start with "WK"
selected_cols = [col for col in merged_df.columns if col.startswith('WK')]

# create a new dataframe called Whole_Program_Total with the selected columns
Whole_Program_Total = merged_df[selected_cols]

In [113]:
merged_df["Program_Total_Hours"]=Whole_Program_Total.sum(axis=1)

In [114]:
# select columns from data where the column names start with "WK"
selected_cols_1 = [col for col in merged_df.columns if col.startswith("Pre_Recorded_Percentage") or col.startswith("SUK_Percentage") or col.startswith("Masterclass_Percentage") or col.startswith("Workshop_Percentage")]

# create a new dataframe called Whole_Program_Total with the selected columns
Whole_Program_Percent = merged_df[selected_cols_1]

In [115]:
merged_df["Program_Percentage"]=Whole_Program_Percent.mean(axis=1)

In [116]:
merged_df=merged_df.round(2)

In [118]:
# Function to clean email addresses
def clean_email(email):

    # Convert to lowercase and remove extra spaces
    cleaned_email = email.lower().strip()
    # Remove patterns like ".com.1"
    cleaned_email = re.sub(r'\.com\.\d+', '.com', cleaned_email)
    return cleaned_email

# Apply the function to the 'email' column
merged_df['Email'] = merged_df['Email'].apply(clean_email)

In [119]:
# Function to rearrange columns
def rearrange_columns(merged_df):
    def extract_week_number(col):
        match = re.search(r'WK(\d+)', col)
        return int(match.group(1)) if match else -1

    wk_columns = sorted([col for col in merged_df.columns if col.startswith('WK')], key=extract_week_number)
    total_columns = [col for col in merged_df.columns if col.endswith('Hours')]
    percentage_columns = [col for col in merged_df.columns if col.endswith('Percentage')]
    other_columns = [col for col in merged_df.columns if col not in wk_columns + total_columns + percentage_columns]
    
    new_column_order = other_columns + wk_columns + total_columns + percentage_columns
    return merged_df[new_column_order]

# Applying the function
merged_df = rearrange_columns(merged_df)

In [120]:
# Function to clean email addresses
def clean_email(email):

    # Convert to lowercase and remove extra spaces
    cleaned_email = email.lower().strip()
    # Remove patterns like ".com.1"
    cleaned_email = re.sub(r'\.com\.\d+', '.com', cleaned_email)
    return cleaned_email

# Apply the function to the 'email' column
merged_df['Email'] = merged_df['Email'].apply(clean_email)

# Adding Genral Info and Exporting

In [ ]:
# Importing data from Genral Info
gi = pd.read_csv(r"")

In [123]:
# Taking columns we want
gi = gi[['Email', 'Name', 'Phone', 'Name_of_College_University','Currently_Pursuing_Degree']].copy()

In [124]:
# Merge with gi
output = pd.merge(gi, merged_df, on='Email', how='outer')

In [ ]:
# Exporting data to Output files
output.to_csv(r"")

# Incubator and attendance monitoring MySQL table

In [126]:
merged_df.columns

Index(['Email', 'WK0_V1', 'WK0_V2', 'WK0_SUK_V', 'WK1_V1', 'WK1_V2', 'WK1_WS1',
       'WK1_Master1', 'WK2_SUK_V', 'WK3_V1', 'WK3_V2', 'WK3_WS2',
       'WK3_Master2', 'WK4_SUK_V', 'WK4_WS3', 'WK5_V1', 'WK5_V2', 'WK5_V3',
       'WK5_WS4', 'WK5_WS5', 'WK5_Master3', 'WK6_WS6', 'WK6_SUK_V', 'WK7_V1',
       'WK7_WS7', 'WK9_V1', 'WK9_V2', 'WK9_WS8', 'WK9_Master4', 'WK10_V1',
       'WK10_V2', 'WK10_V3', 'WK10_V4', 'WK10_WS9', 'WK10_Master5',
       'WK10_WS10', 'WK10_SUK_V', 'Pre_Recorded_Total_Hours',
       'SUK_Total_Hours', 'Masterclass_Total_Hours', 'Workshop_Total_Hours',
       'Program_Total_Hours', 'Pre_Recorded_Percentage', 'SUK_Percentage',
       'Masterclass_Percentage', 'Workshop_Percentage', 'Program_Percentage'],
      dtype='object')

In [128]:
merged_df.columns

Index(['Email', 'WK0_V1', 'WK0_V2', 'WK0_SUK_V', 'WK1_V1', 'WK1_V2', 'WK1_WS1',
       'WK1_Master1', 'WK2_SUK_V', 'WK3_V1', 'WK3_V2', 'WK3_WS2',
       'WK3_Master2', 'WK4_SUK_V', 'WK4_WS3', 'WK5_V1', 'WK5_V2', 'WK5_V3',
       'WK5_WS4', 'WK5_WS5', 'WK5_Master3', 'WK6_WS6', 'WK6_SUK_V', 'WK7_V1',
       'WK7_WS7', 'WK9_V1', 'WK9_V2', 'WK9_WS8', 'WK9_Master4', 'WK10_V1',
       'WK10_V2', 'WK10_V3', 'WK10_V4', 'WK10_WS9', 'WK10_Master5',
       'WK10_WS10', 'WK10_SUK_V', 'Pre_Recorded_Total_Hours',
       'SUK_Total_Hours', 'Masterclass_Total_Hours', 'Workshop_Total_Hours',
       'Program_Total_Hours', 'Pre_Recorded_Percentage', 'SUK_Percentage',
       'Masterclass_Percentage', 'Workshop_Percentage', 'Program_Percentage'],
      dtype='object')

In [129]:
oldcol

['Email',
 'Video ! : Why women in STEM. What has Kalpana program achieved until now? - 62c7d78d0cf273488220b8d4',
 'Video @ : What do we expect from our Kalpana fellows? - 62c7d10e0cf273488220b5bf',
 'SUK + : Program Inauguration & Icebreaker 7 Sept - 66dc3e605ae308684ff5e726',
 'Video # : Goal setting - Part 1 - 6529ffc5e4b0a564951cb119',
 'Video $ : Goal setting - Part 2 - 6529ffd4e4b08a0cfb8d0628',
 'Workshop @ : Setting Aspirational Goals, Thursday, Sept 12, 3-4 PM - 66de9bf79027bf0127e793ec',
 'Master Class # : Searching and Securing Internships - Friday, Sept 13, 3-4 PM - 66e125d35e916d19ae3df9b1',
 'SUK ! : Speak Up Kalpana session, Friday, Sept 20, 3-4 PM - 66e13e6e91e40e365df6fdd4',
 'Video % : What is SMART goal ? - 62b3fded0cf2f5120a60c932',
 'Video ^ : What is the difference between a goal and a SMART goal ? - 62b3fe320cf2f5120a60c946',
 'Workshop $ : Setting SMART Goals, Thursday, Sept 26, 3-4 PM - 66e13f3476aec348317d2f82',
 'Master Class $ : Acing your CV and Resume - F

In [130]:
oldcol = [col for col in oldcol if not col.startswith("Week")]


In [131]:
oldcol

['Email',
 'Video ! : Why women in STEM. What has Kalpana program achieved until now? - 62c7d78d0cf273488220b8d4',
 'Video @ : What do we expect from our Kalpana fellows? - 62c7d10e0cf273488220b5bf',
 'SUK + : Program Inauguration & Icebreaker 7 Sept - 66dc3e605ae308684ff5e726',
 'Video # : Goal setting - Part 1 - 6529ffc5e4b0a564951cb119',
 'Video $ : Goal setting - Part 2 - 6529ffd4e4b08a0cfb8d0628',
 'Workshop @ : Setting Aspirational Goals, Thursday, Sept 12, 3-4 PM - 66de9bf79027bf0127e793ec',
 'Master Class # : Searching and Securing Internships - Friday, Sept 13, 3-4 PM - 66e125d35e916d19ae3df9b1',
 'SUK ! : Speak Up Kalpana session, Friday, Sept 20, 3-4 PM - 66e13e6e91e40e365df6fdd4',
 'Video % : What is SMART goal ? - 62b3fded0cf2f5120a60c932',
 'Video ^ : What is the difference between a goal and a SMART goal ? - 62b3fe320cf2f5120a60c946',
 'Workshop $ : Setting SMART Goals, Thursday, Sept 26, 3-4 PM - 66e13f3476aec348317d2f82',
 'Master Class $ : Acing your CV and Resume - F

In [132]:
#data.columns = oldcol[:len(data.columns)]
# Now, let's say you want to change the column names of 'old_dataset' based on 'filtered_column_names'
num_columns_to_rename = min(len(merged_df.columns), len(oldcol))
print(num_columns_to_rename)

37


In [133]:
merged_df.rename(columns=dict(zip(merged_df.columns[:num_columns_to_rename], oldcol[:num_columns_to_rename])), inplace=True)


In [134]:
merged_df.columns

Index(['Email',
       'Video ! : Why women in STEM. What has Kalpana program achieved until now? - 62c7d78d0cf273488220b8d4',
       'Video @ : What do we expect from our Kalpana fellows? - 62c7d10e0cf273488220b5bf',
       'SUK + : Program Inauguration & Icebreaker 7 Sept - 66dc3e605ae308684ff5e726',
       'Video # : Goal setting - Part 1 - 6529ffc5e4b0a564951cb119',
       'Video $ : Goal setting - Part 2 - 6529ffd4e4b08a0cfb8d0628',
       'Workshop @ : Setting Aspirational Goals, Thursday, Sept 12, 3-4 PM - 66de9bf79027bf0127e793ec',
       'Master Class # : Searching and Securing Internships - Friday, Sept 13, 3-4 PM - 66e125d35e916d19ae3df9b1',
       'SUK ! : Speak Up Kalpana session, Friday, Sept 20, 3-4 PM - 66e13e6e91e40e365df6fdd4',
       'Video % : What is SMART goal ? - 62b3fded0cf2f5120a60c932',
       'Video ^ : What is the difference between a goal and a SMART goal ? - 62b3fe320cf2f5120a60c946',
       'Workshop $ : Setting SMART Goals, Thursday, Sept 26, 3-4 PM - 66

In [135]:
# Split the columns by ':' and keep only the first part
merged_df.columns = merged_df.columns.str.split (':').str [0]

In [136]:
merged_df.columns

Index(['Email', 'Video ! ', 'Video @ ', 'SUK + ', 'Video # ', 'Video $ ',
       'Workshop @ ', 'Master Class # ', 'SUK ! ', 'Video % ', 'Video ^ ',
       'Workshop $ ', 'Master Class $ ', 'SUK @ ', 'Workshop !! ', 'Video $? ',
       'Video %+ ', 'Video %! ', 'Workshop !# ', 'Workshop % ',
       'Master Class ! ', 'Workshop ^ ', 'SUK # ', 'Video %@ ', 'Workshop & ',
       'Video %# ', 'Video %$ ', 'Workshop * ', 'Master Class % ', 'Video & ',
       'Video * ', 'Video ? ', 'Video !+ ', 'Workshop ? ', 'Master Class !^ ',
       'Workshop !@ ', 'SUK $ ', 'Pre_Recorded_Total_Hours', 'SUK_Total_Hours',
       'Masterclass_Total_Hours', 'Workshop_Total_Hours',
       'Program_Total_Hours', 'Pre_Recorded_Percentage', 'SUK_Percentage',
       'Masterclass_Percentage', 'Workshop_Percentage', 'Program_Percentage'],
      dtype='object')

In [137]:
merged_df.columns = merged_df.columns.str.strip()


In [138]:
def rename_column(col):
    # Split the column name by space
    parts = col.split()
    special_chars = {'!': '1', '@': '2', '#': '3', '$': '4', '%': '5', '^': '6', '&': '7', '*': '8', '?': '9', '+': '0'}
    
    # If the first part is 'SUK', add the number corresponding to the last character
    if parts[0] == 'SUK':
        return parts[0] + special_chars.get(parts[1][-1], '0')
    
    # If the first part is 'Video', use the prefix 'VID' and the numbers corresponding to the last two characters
    elif parts[0] == 'Video':
        last_char = parts[1][-1]
        second_last_char = parts[1][-2] if len(parts[1]) > 1 else '+'
        return 'VID' + special_chars.get(second_last_char, '0') + special_chars.get(last_char, '0')

    # If the first part is 'Workshop', use the prefix 'VID' and the numbers corresponding to the last two characters
    elif parts[0] == 'Workshop':
        last_char = parts[1][-1]
        second_last_char = parts[1][-2] if len(parts[1]) > 1 else '+'
        return 'WS' + special_chars.get(second_last_char, '0') + special_chars.get(last_char, '0')
    
    # If the first part contains 'Master' and 'Class' is present, use the prefix 'MC' and the numbers corresponding to the last two characters
    elif 'Master' in parts and 'Class' in parts:
        index = parts.index('Master')
        last_char = parts[index + 2][-1]
        second_last_char = parts[index + 2][-2] if len(parts[index + 2]) > 1 else '+'
        return 'MC' + special_chars.get(second_last_char, '0') + special_chars.get(last_char, '0')
    
    # Otherwise, return the column name as it is
    else:
        return col


In [139]:
# Rename the column names using the rename_column function
merged_df.rename(columns=rename_column, inplace=True)

In [140]:
merged_df.columns

Index(['Email', 'VID01', 'VID02', 'SUK0', 'VID03', 'VID04', 'WS02', 'MC03',
       'SUK1', 'VID05', 'VID06', 'WS04', 'MC04', 'SUK2', 'WS11', 'VID49',
       'VID50', 'VID51', 'WS13', 'WS05', 'MC01', 'WS06', 'SUK3', 'VID52',
       'WS07', 'VID53', 'VID54', 'WS08', 'MC05', 'VID07', 'VID08', 'VID09',
       'VID10', 'WS09', 'MC16', 'WS12', 'SUK4', 'Pre_Recorded_Total_Hours',
       'SUK_Total_Hours', 'Masterclass_Total_Hours', 'Workshop_Total_Hours',
       'Program_Total_Hours', 'Pre_Recorded_Percentage', 'SUK_Percentage',
       'Masterclass_Percentage', 'Workshop_Percentage', 'Program_Percentage'],
      dtype='object')

In [141]:
merged_df.shape

(639, 47)

In [142]:
merged_df.head()

,Email,VID01,VID02,SUK0,VID03,VID04,WS02,MC03,SUK1,VID05,VID06,WS04,MC04,SUK2,WS11,VID49,VID50,VID51,WS13,WS05,MC01,WS06,SUK3,VID52,WS07,VID53,VID54,WS08,MC05,VID07,VID08,VID09,VID10,WS09,MC16,WS12,SUK4,Pre_Recorded_Total_Hours,SUK_Total_Hours,Masterclass_Total_Hours,Workshop_Total_Hours,Program_Total_Hours,Pre_Recorded_Percentage,SUK_Percentage,Masterclass_Percentage,Workshop_Percentage,Program_Percentage
0,monu1288@gmail.com,0.00,0.00,0.00,0.00,0.00,0.08,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.01,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.09,0.09,0.00,0.02,0.03,0.97,0.25
1,sreejith.sreenivasan@vigyanshaala.com,0.00,0.00,0.05,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.04,0.05,0.00,0.00,0.08,5.22,1.18,0.00,0.00,1.60
2,spjaybhaye01@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,salini.1405@gmail.com,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.02,0.00,0.00,0.02,0.16,0.39,0.00,0.00,0.14
4,ds9284751@gmail.com,0.08,0.08,2.88,0.08,0.11,1.01,2.68,2.29,0.24,0.08,1.2,2.43,1.99,1.12,0.05,0.08,0.16,1.23,1.62,1.64,1.0,2.03,0.19,1.15,0.27,0.1,0.18,1.12,0.04,0.1,0.13,0.0,1.85,0.0,0.0,0.0,1.78,9.18,7.87,10.36,29.19,93.75,100.00,80.00,90.93,91.17


In [143]:
mask = merged_df.columns.duplicated().sum()
print(mask)

0


In [ ]:
# Exporting data to Output files
merged_df.to_csv(r"")

# Storing data on MySQL

In [ ]:
# Establish connection
conn= msql.connect(host='',user='',password="",database="",auth_plugin='')
cursor =conn.cursor() 

In [146]:
# Get the existing columns in the database
cursor.execute("SHOW COLUMNS FROM 08_incubator_and_attendance_monitoring")
existing_columns = [col[0] for col in cursor.fetchall()]

# Define the column name before which the new column should be added
target_column = 'Pre_Recorded_Total_Hours'

# Check if any new columns exist in the dataframe but not in the database
new_columns = [col for col in merged_df.columns if col not in existing_columns]
if new_columns:
    # Add new columns to the database before the target column
    for col in reversed(new_columns):
        if col not in existing_columns:
            # Get the index of the target column
            target_column_index = existing_columns.index(target_column)
            # Set the data type based on whether the column name starts with Comment
            data_type = "varchar(50)" 
            alter_query = f"ALTER TABLE 08_incubator_and_attendance_monitoring ADD COLUMN {col} {data_type} AFTER {existing_columns[target_column_index - 1]}"
            cursor.execute(alter_query)
            existing_columns.insert(target_column_index - 1, col)

In [147]:
# Your existing code for inserting data into the database table
for i, row in merged_df.iterrows():
    row = [None if isinstance(val, float) and math.isnan(val) else val for val in row] # replace "nan" values with None
    columns = ','.join(merged_df.columns)
    placeholders = ','.join(['%s']*len(row))
    # Construct the INSERT query with ON DUPLICATE KEY UPDATE clause
    query = f"INSERT INTO 08_incubator_and_attendance_monitoring ({columns}) VALUES ({placeholders}) ON DUPLICATE KEY UPDATE "
    query += ", ".join([f"{col}=VALUES({col})" for col in merged_df.columns if col != 'Email'])
    # Execute the query
    cursor.execute(query, tuple(row))

In [148]:
# Commit the transaction
conn.commit()

In [149]:
# Close the connection
cursor.close()
conn.close()


# Storing Data in Supabase

In [ ]:
# Supabase URL and API key
url = ''
api_key = ''


# Headers for the request
headers = {
    'apikey': api_key,
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json',
    'Prefer': 'resolution=merge-duplicates'  # Enable upsert functionality
}


### 03_enroll_date Table

In [151]:
# Rename the column 'old_column_name' to 'new_column_name'
Enroll.rename(columns={'Enroll Date': 'Incubator'}, inplace=True)

In [152]:
Enroll.columns

Index(['Email', 'Incubator'], dtype='object')

In [153]:
table_name = '03_enroll_date'

# Batch size for upserting
batch_size = 1000  # You can adjust this value based on your performance needs

# List to store rows before sending them in batches
batch_data = []

# Iterate through each row in the DataFrame
for i, row in Enroll.iterrows():
    # Replace NaN values with None
    row = [None if isinstance(val, float) and math.isnan(val) else val for val in row]
    # Convert row to a dictionary
    row_dict = dict(zip(Enroll.columns, row))
    
    # Add the row to the batch
    batch_data.append(row_dict)
    
    # If the batch size is reached, send the data
    if len(batch_data) >= batch_size:
        # Send a batch of rows
        response = requests.post(f'{url}/rest/v1/{table_name}', headers=headers, json=batch_data)
        
        # Check response
        if response.status_code in [200, 201]:
            print(f'Batch of {len(batch_data)} rows upserted successfully')
        else:
            print(f'Failed to upsert batch: {response.status_code}, {response.text}')
        
        # Clear the batch after sending
        batch_data = []

# Send any remaining rows in the last batch
if batch_data:
    response = requests.post(f'{url}/rest/v1/{table_name}', headers=headers, json=batch_data)
    
    if response.status_code in [200, 201]:
        print(f'Final batch of {len(batch_data)} rows upserted successfully')
    else:
        print(f'Failed to upsert final batch: {response.status_code}, {response.text}')


Final batch of 503 rows upserted successfully


### 02_payment_details Table

In [154]:
# Rename the column 'old_column_name' to 'new_column_name'
Payment.rename(columns={'Assigned Through': 'Incubator_Fee'}, inplace=True)

In [155]:
table_name = '02_payment_details'

# Batch size for upserting
batch_size = 1000  # You can adjust this value based on your performance needs

# List to store rows before sending them in batches
batch_data = []

# Iterate through each row in the DataFrame
for i, row in Payment.iterrows():
    # Replace NaN values with None
    row = [None if isinstance(val, float) and math.isnan(val) else val for val in row]
    # Convert row to a dictionary
    row_dict = dict(zip(Payment.columns, row))
    
    # Add the row to the batch
    batch_data.append(row_dict)
    
    # If the batch size is reached, send the data
    if len(batch_data) >= batch_size:
        # Send a batch of rows
        response = requests.post(f'{url}/rest/v1/{table_name}', headers=headers, json=batch_data)
        
        # Check response
        if response.status_code in [200, 201]:
            print(f'Batch of {len(batch_data)} rows upserted successfully')
        else:
            print(f'Failed to upsert batch: {response.status_code}, {response.text}')
        
        # Clear the batch after sending
        batch_data = []

# Send any remaining rows in the last batch
if batch_data:
    response = requests.post(f'{url}/rest/v1/{table_name}', headers=headers, json=batch_data)
    
    if response.status_code in [200, 201]:
        print(f'Final batch of {len(batch_data)} rows upserted successfully')
    else:
        print(f'Failed to upsert final batch: {response.status_code}, {response.text}')


Final batch of 503 rows upserted successfully


### 15_batch_lastlogin_startdate

In [156]:
# Rename the column 'old_column_name' to 'new_column_name'
Batch.rename(columns={'Last Login': 'Incubator_Last_Login'}, inplace=True)
Batch.rename(columns={'Start Date': 'Incubator_Last_Login'}, inplace=True)

In [158]:
table_name = '15_batch_lastlogin_startdate'

# Batch size for upserting
batch_size = 1000  # You can adjust this value based on your performance needs

# List to store rows before sending them in batches
batch_data = []

# Iterate through each row in the DataFrame
for i, row in Batch.iterrows():
    # Replace NaN values with None
    row = [None if isinstance(val, float) and math.isnan(val) else val for val in row]
    # Convert row to a dictionary
    row_dict = dict(zip(Batch.columns, row))
    
    # Add the row to the batch
    batch_data.append(row_dict)
    
    # If the batch size is reached, send the data
    if len(batch_data) >= batch_size:
        # Send a batch of rows
        response = requests.post(f'{url}/rest/v1/{table_name}', headers=headers, json=batch_data)
        
        # Check response
        if response.status_code in [200, 201]:
            print(f'Batch of {len(batch_data)} rows upserted successfully')
        else:
            print(f'Failed to upsert batch: {response.status_code}, {response.text}')
        
        # Clear the batch after sending
        batch_data = []

# Send any remaining rows in the last batch
if batch_data:
    response = requests.post(f'{url}/rest/v1/{table_name}', headers=headers, json=batch_data)
    
    if response.status_code in [200, 201]:
        print(f'Final batch of {len(batch_data)} rows upserted successfully')
    else:
        print(f'Failed to upsert final batch: {response.status_code}, {response.text}')


Final batch of 503 rows upserted successfully


### 08_incubator_and_attendance_monitoring Table

In [159]:
table_name = '08_incubator_and_attendance_monitoring'

# Batch size for upserting
batch_size = 1000  # You can adjust this value based on your performance needs

# List to store rows before sending them in batches
batch_data = []

# Iterate through each row in the DataFrame
for i, row in merged_df.iterrows():
    # Replace NaN values with None
    row = [None if isinstance(val, float) and math.isnan(val) else val for val in row]
    # Convert row to a dictionary
    row_dict = dict(zip(merged_df.columns, row))
    
    # Add the row to the batch
    batch_data.append(row_dict)
    
    # If the batch size is reached, send the data
    if len(batch_data) >= batch_size:
        # Send a batch of rows
        response = requests.post(f'{url}/rest/v1/{table_name}', headers=headers, json=batch_data)
        
        # Check response
        if response.status_code in [200, 201]:
            print(f'Batch of {len(batch_data)} rows upserted successfully')
        else:
            print(f'Failed to upsert batch: {response.status_code}, {response.text}')
        
        # Clear the batch after sending
        batch_data = []

# Send any remaining rows in the last batch
if batch_data:
    response = requests.post(f'{url}/rest/v1/{table_name}', headers=headers, json=batch_data)
    
    if response.status_code in [200, 201]:
        print(f'Final batch of {len(batch_data)} rows upserted successfully')
    else:
        print(f'Failed to upsert final batch: {response.status_code}, {response.text}')


Final batch of 639 rows upserted successfully
